In [ ]:
%%capture
%run preprocessing.ipynb

In [1]:
import numpy as np


#loading data 
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [7]:
import numpy as np
from sklearn import model_selection
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset 
from torch import flatten
from torch import unsqueeze

class Classifier(nn.Module):
    def __init__(self, dropout):

        super(Classifier, self).__init__()
        self.dropout = nn.Dropout(dropout)

        # First layer
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=20, kernel_size=2)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride = 2)

        # Second layer
        self.conv2 = nn.Conv1d(in_channels=20, out_channels=40, kernel_size=2)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride = 2)

        # Third layer (fully connected)
        self.fc1 = nn.Linear(in_features=40, out_features=20)
        self.relu3 = nn.ReLU()

        # Softmax Classifier
        self.fc2 = nn.Linear(in_features=20, out_features=3)
        self.softmax = nn.Softmax(dim=1)


    # specifying the flow of data through the layers
    def forward(self, x):
        # Pass through conv1
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # Pass through conv2
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # Flatten the output before fully connected layers
        x = torch.flatten(x, 1)

        # Pass through the fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)
        output = self.softmax(x)

        return output  # Return the output
    
# Load data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy') - 1 # subtract 1 to make the labels 0, 1, 2 
y_test = np.load('y_test.npy') -1    # same thing here

# Reshape your data to simulate a single-channel image
#X_train_reshaped = X_train.reshape(-1, 1, 7, 1)  # Reshape training data
#X_test_reshaped = X_test.reshape(-1, 1, 7, 1)    # Reshape test data

# Assuming X_train and X_test have shapes (num_samples, num_features)
X_train_tensor = torch.tensor(X_train.transpose(0, 1), dtype=torch.float32).unsqueeze(1)  # Shape: (num_features, num_samples) -> (num_samples, 1, num_features)
X_test_tensor = torch.tensor(X_test.transpose(0, 1), dtype=torch.float32).unsqueeze(1)    # Shape: (num_features, num_samples) -> (num_samples, 1, num_features)


# Convert your data into PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Convert your data into PyTorch tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create Tensor datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # amount of data to be loaded each time
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

# Instantiate the more complex CNN
model = Classifier(0.67)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    all_labels = []
    all_predictions = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)   # get the index of the max log-probability
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

# Print metrics
print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 Score: {}'.format(f1))

Accuracy: 0.9757051745676902
Precision: 0.9757844502444736
Recall: 0.9757051745676902
F1 Score: 0.9756593890144857
